# Cross validation

In [ ]:
import os
import sys
from fitting import FitModel
import numpy as np
import matplotlib.pyplot as plt
import json
import glob

In [ ]:
def get_forces(fit_data, values, args):
    fit_data.init_potential(values, args)
    ip_forces = fit_data.get_forces()
    dft_forces = fit_data.expected_forces()
    return dft_forces, ip_forces

def chi_squared_error(dft_forces, ip_forces):
    return np.sum((dft_forces - ip_forces)**2)/ dft_forces.size

def create_directory(head_directory_name, structure_number):
    directory = os.path.join(head_directory_name, str(structure_number))
    os.makedirs(directory)
    return directory

### Set up parameters for LiNiO2 with core-shell O-O

In [ ]:
params = {}
params['core_shell'] = { 'Li': False, 'Ni': False, 'O': True }
params['charges'] = {'Li': +1.0,
                     'Ni': +3.0,
                     'O': {'core':  -2.0, #+0.960,
                           'shell': 0.0}} #-2.960}}
params['masses'] = {'Li': 6.941,
                    'Ni': 58.6934,
                    'O': {'core': 14.3991,
                          'shell': 1.5999} }
params['cs_springs'] = {'O-O' : [20.0, 0.0]}

distribution = {}
distribution['Li-O'] = {'bpp' : [663.111, 0.119, 0.0],
                        'sd' : [80, 0.01, 0.01]}
distribution['Ni-O'] = {'bpp' : [1393.540, 0.218, 0.000],
                        'sd'  : [80, 0.01, 0.01]}
distribution['O-O'] = {'bpp' : [25804.807, 0.284, 0.0],
                       'sd'  : [200, 0.01, 5]}

### Define the directory paths and names/number of structures

In [ ]:
# Create cross validation directory
head_directory_name = '1_structure_fits'
cv_directory_name = 'cross_validation'
head_output_directory = create_directory(head_directory_name, cv_directory_name)


# Define paths to poscar/outcar directories
poscar_directory = os.path.join('poscars','thermos')
outcar_directory = os.path.join('outcars','thermos')

# Define number of structures
number_of_structures = 15

### Calculates each structures potential with every other structure, returning the forces and error

In [ ]:
for potential_file in sorted(glob.glob('{}/*/potentials.json'.format(head_directory_name,cv_directory_name))): #change 1 to * for all cross-validations
    with open(potential_file, 'r') as f:
        potentials = json.load(f)
    structure_num = int(potential_file.replace('/potentials.json', '').replace('1_structure_fits/',''))
    include_labels = list(potentials.keys())
    include_values = list(potentials.values())
    indv_output_directory = create_directory(head_output_directory, 'p{}'.format(structure_num))
    for structure in range(number_of_structures):
        if (structure+1) != structure_num:
            os.system('cp {}/POSCAR{} {}/{}'.format(poscar_directory, structure+1, 'poscars', 'POSCAR1'))
            os.system('cp {}/OUTCAR{} {}/{}'.format(outcar_directory, structure+1, 'outcars', 'OUTCAR1'))
            fit_data = FitModel.collect_info(params, distribution, supercell=[2,2,2])
            dft_forces, ip_forces = get_forces(fit_data, include_values, include_labels)
            error = chi_squared_error(dft_forces, ip_forces)      
#             print('{}/p{}-s{}_dft_forces.dat'.format(indv_output_directory, structure_num, structure+1))
            np.savetxt('{}/p{}-s{}_dft_forces.dat'.format(indv_output_directory, structure_num, structure+1), dft_forces, fmt='%.10e', delimiter=' ')
            np.savetxt('{}/p{}-s{}_ip_forces.dat'.format(indv_output_directory, structure_num, structure+1), ip_forces, fmt='%.10e', delimiter=' ')
            with open('{}/p{}-s{}_error.dat'.format(indv_output_directory, structure_num, structure+1), 'w') as f:
                f.write(str(error))